# Displays the Network of borrowed terms for Chemistry ontologies 
* Ontologies listed in listed in [ontologies_details.yml](ontologies_details.yml)
* Queries made to OLS API from <https://terminology.nfdi4chem.de/> & <http://www.ebi.ac.uk/ols>
   * OLS API documentation <https://www.ebi.ac.uk/ols/docs/api>
* Uses [pyvis](https://pyvis.readthedocs.io/en/latest/index.html) Visualization Library

In [12]:
import requests
import yaml
from time import sleep
from pprint import pprint

############
# Gathers all terms and borrowed Ontologies employed by Chemistry ontologies, listed in ontologies_details.yml
# var chem_ontos_terms - is dictionary holding ontology terms organized by ontology id/prefix
# var all_ontos - is dictionary where holding info on ontologies employed by chemestry ontologies, organized by ontology id/prefix
############

'''
chem_ontos_terms structure

'RXNO': { 
    'RXNO:0000563': {'id': 'RXNO:0000563',
                    'iri': 'http://purl.obolibrary.org/obo/RXNO_0000563',
                    'label': 'obsolete: chloro Heck-type reaction/Sandmeyer '
                            'fluorination',
                    'onto_prefix': 'RXNO'},
    },
'CHIRO': ...
'''


def yaml2dict(path):
    with open(path, 'r') as yaml_f:
        yaml_content = yaml_f.read()
        yaml_dict = yaml.safe_load(yaml_content)
    return yaml_dict

def get_all_ontology_terms(api_url, size, page, terms):
    '''
    return dict with all terms of an ontology, 
    each entry with keys: iri, id, label, onto_prefix
    '''
    api_url_w_params = f'{api_url}?page={page}&size={size}'
    try:
        request = requests.get(api_url_w_params)
    except requests.exceptions.Timeout:
        sleep(1)
        request = requests.get(api_url_w_params)
    response = request.json()
    page_numb = response['page']['number']
    page_total  = response['page']['totalPages']
    # print(f'Current page:{page_numb}  Total pages:{page_total}')
    if '_embedded' in response.keys():
        terms.update({term['obo_id']: {'iri': term['iri'], 'id': term['obo_id'], 'label': term['label'], 'onto_prefix': term['obo_id'].split(':')[0]} for term in response['_embedded']['terms'] if term['obo_id']})
        get_all_ontology_terms(api_url=api_url, size=size, page=page+1, terms=terms)
    return terms

def get_onto_info(onto_prefix):
    onto_api_url =  f'{tib_ols_api}ontologies/{onto_prefix}'
    onto_request = requests.get(onto_api_url)
    if onto_request.status_code == 404:
        onto_api_url =  f'{ebispot_ols_api}ontologies/{onto_prefix}'
        onto_request = requests.get(onto_api_url)
        if onto_request.status_code == 404:
            # print(f'No ontology info in OLS for {onto_prefix}')
            onto_response = {}     
        else:
            # print(f'getting {onto_prefix} from EBISPOT OLS')
            onto_response = onto_request.json() 
    else:
        # print(f'getting {onto_prefix} from TIB OLS')
        onto_response = onto_request.json() 
    return onto_response


tib_ols_api = 'https://service.tib.eu/ts4tib/api/'
ebispot_ols_api = 'http://www.ebi.ac.uk/ols/api/'
chem_ontos = yaml2dict(path='ontologies_details.yml')
chem_ontos_ids = [onto['id'].upper() for onto in chem_ontos]
chem_ontos_terms = {}

for onto in chem_ontos:
    print(f"Ontology: {onto['id']}")
    all_onto_terms = get_all_ontology_terms(api_url=f'{tib_ols_api}ontologies/{onto["id"]}/terms', size=100, page=0, terms={})
    print(f"Ontology: {onto['id']} has {len(all_onto_terms)} terms")
    chem_ontos_terms[onto['id'].upper()] = all_onto_terms
    ontos_used_in_ontos = set([v['onto_prefix'] for v in all_onto_terms.values()])
    print(f"Ontology: {onto['id']} uses the {', '.join(ontos_used_in_ontos)} ontologies")
    print(f'\n{"*"*20}\n')

print(chem_ontos_terms.keys())



Ontology: RXNO
Ontology: RXNO has 901 terms
Ontology: RXNO uses the BFO, MOP, CHEBI, RXNO, IAO, OBI ontologies

********************

Ontology: MOP
Ontology: MOP has 3682 terms
Ontology: MOP uses the BFO, MOP, CHEBI, REX, RXNO ontologies

********************

Ontology: CHEMINF
Ontology: CHEMINF has 832 terms
Ontology: CHEMINF uses the BFO, APOLLO_SV, CHEBI, GAZ, CDK, PR, UO, GO, IAO, PATO, CHEMINF, OBI ontologies

********************

Ontology: CHMO
Ontology: CHMO has 3084 terms
Ontology: CHMO uses the OBCS, BFO, CHEBI, REX, FIX, MS, CHMO, IAO, OBI ontologies

********************

Ontology: CHIRO
Ontology: CHIRO has 13284 terms
Ontology: CHIRO uses the BFO, CHEBI, ENVO, MP, HP, PR, NCBITaxon, GO, RO ontologies

********************

dict_keys(['RXNO', 'MOP', 'CHEMINF', 'CHMO', 'CHIRO'])


In [2]:
from pyvis.network import Network

def draw_onto_relations(net, target_onto, options, width, height):
    # chem_ontos_terms => onto dict
    net.width = width
    net.height = height
    net.add_node(target_onto, label=target_onto, color="#6BFFF5", borderWidth=0, shape="circle", size=1500) # src onto
    for onto_term_k, dest_onto_term in chem_ontos_terms[target_onto].items():
        # add ontologies nodes
        onto =  dest_onto_term['onto_prefix']
        color='#ffff00'
        net.add_node(onto, label=onto,  color=color, borderWidth=0, shape="circle", size=500) # dest onto
        if dest_onto_term['onto_prefix'] != target_onto:
            net.add_edge(target_onto, dest_onto_term['onto_prefix'], color='#ACACAC', title=f"{dest_onto_term['id']} - {dest_onto_term['label']}")
    net.set_options(options)
    return net
    
graph_opts = '''
    var options = {
    "interaction": {
        "navigationButtons": true
    },
    "physics": {
        "repulsion": {
        "centralGravity": 2.9,
        "springLength": 140,
        "nodeDistance": 255,
        "damping": 0.08
        },
        "maxVelocity": 3,
        "minVelocity": 0.13,
        "solver": "repulsion",
        "timestep": 0.48
    },
    "nodes": {
        "font": {
        "size": 16
        }
    },
    "edges": {
        "arrows": {
        "to": {
            "enabled": true,
            "scaleFactor": 0
        }
    }
  }
}
'''


In [5]:
ontology = chem_ontos[0]['id']
net = draw_onto_relations(
    net=Network(directed=True, bgcolor='#0A0A0A', notebook=True, heading=f'{ontology} Borrowed Terms'), 
    target_onto=ontology, 
    options=graph_opts,
    width='1000px',
    height='750px')
net.show(f'{ontology}.html')
# net.save_graph(f'{target_onto}.html')



In [6]:
ontology = chem_ontos[1]['id']
net = draw_onto_relations(
    net=Network(directed=True, bgcolor='#0A0A0A', notebook=True, heading=f'{ontology} Borrowed Terms'), 
    target_onto=ontology, 
    options=graph_opts,
    width='1000px',
    height='750px')
net.show(f'{ontology}.html')

In [7]:
ontology = chem_ontos[2]['id']
net = draw_onto_relations(
    net=Network(directed=True, bgcolor='#0A0A0A', notebook=True, heading=f'{ontology} Borrowed Terms'), 
    target_onto=ontology, 
    options=graph_opts,
    width='1000px',
    height='750px')
net.show(f'{ontology}.html')

In [9]:
ontology = chem_ontos[3]['id']
net = draw_onto_relations(
    net=Network(directed=True, bgcolor='#0A0A0A', notebook=True, heading=f'{ontology} Borrowed Terms'), 
    target_onto=ontology, 
    options=graph_opts,
    width='1000px',
    height='750px')
net.show(f'{ontology}.html')

In [10]:
ontology = chem_ontos[4]['id']
net = draw_onto_relations(
    net=Network(directed=True, bgcolor='#0A0A0A', notebook=True, heading=f'{ontology} Borrowed Terms'), 
    target_onto=ontology, 
    options=graph_opts,
    width='1000px',
    height='750px')
net.show(f'{ontology}.html')

KeyError: 'CHIRO'